# スイープジョブでハイパーパラメータを調整する

機械学習アルゴリズムの中には、ハイパーパラメータ（学習に影響を与えるが、学習データ自体からは決定できないパラメータ値）を必要とするものがたくさんある。例えば、ロジスティック回帰モデルを学習する場合、正則化率のハイパーパラメータを使用してモデルの偏りを打ち消すことができます。また、畳み込みニューラルネットワークを学習する場合、学習率やバッチサイズのようなハイパーパラメータを使用して、重みの調整方法やミニバッチで処理されるデータ項目の数をそれぞれ制御することができます。ハイパーパラメータ値の選択は、学習済みモデルのパフォーマンスや学習にかかる時間に大きく影響する可能性があり、最適解を見つけるために複数の組み合わせを試す必要があることがよくあります。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml**パッケージの最新版が必要です。以下のセルを実行して、インストールされていることを確認してください。

> 注**：
> もし **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールしてください。

In [2]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.21.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## ワークスペースに接続する

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、サブスクリプションID、リソースグループ名、ワークスペース名といった識別子のパラメータが必要だ。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、デフォルト値を使用してワークスペースに接続できる。

In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## 学習スクリプトの作成
ハイパーパラメータチューニングは、機械学習モデルを訓練したいが入力パラメータを変えたい場合に最適です。アルゴリズムのハイパーパラメータの1つを表す入力パラメータを期待する学習スクリプトを作成する必要があります。

以下のセルを実行して、**src** フォルダーとトレーニングスクリプトを作成します。

トレーニングスクリプトは2つの入力パラメータを必要とすることに注意してください：

- training_data`には文字列を指定します。training_data`には文字列を指定します。
- reg_rate`は数値を指定しますが、デフォルト値は`0.01`です。この入力パラメータはハイパーパラメータのチューニングに使用する。

In [5]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [7]:
%%writefile $script_folder/train.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # read data
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# function that splits the data
def split_data(df):
    print("Splitting data...")
    X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness',
    'SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

    return X_train, X_test, y_train, y_test

# function that trains the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    mlflow.log_param("Regularization rate", reg_rate)
    print("Training model...")
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    return model

# function that evaluates the model
def eval_model(model, X_test, y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)
    mlflow.log_metric("training_accuracy_score", acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))
    mlflow.log_metric("AUC", auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    mlflow.log_artifact("ROC-Curve.png")    

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Overwriting src/train.py


## コマンドジョブの設定と実行

糖尿病を予測する分類モデルをトレーニングするために以下のセルを実行します。モデルは**src**フォルダにある**train.py**スクリプトを実行することで学習されます。学習データとして、登録されている `diabetes-data` データアセットを使用します。

- code`: 実行するスクリプトを含むフォルダを指定します。
- command`: 具体的に何を実行するかを指定する。
- 環境`: コマンドを実行する前に計算機にインストールするパッケージを指定する。
- compute`: コマンドの実行に使用するコンピュートを指定する。
- display_name`: 個々のジョブの名前。
- experiment_name`: ジョブが属する実験の名前。

コマンドジョブは正則化率 `0.1` でトレーニングスクリプトを一度だけ実行することに注意してください。ハイパーパラメータを調整するためにスイープジョブを実行する前に、コマンドジョブでスクリプトが期待通りに動作するかをテストするのがベストプラクティスです。

In [8]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.0 MBs): 100%|████████

Monitor your job at https://ml.azure.com/runs/musing_branch_xw2ht66r4x?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


## Define the search space

コマンド・ジョブが正常に完了したら、掃引ジョブを設定して実行できます。

まず、ハイパーパラメータの探索空間を指定します。3つのモデルをそれぞれ異なる正則化率（`0.01`, `0.1`, `1`）で学習させるためには、`Choice`ハイパーパラメータで探索空間を定義します。

[ハイパーパラメータチューニングの選択](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-tune-hyperparameters?view=azureml-api-2)

In [12]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## スイープジョブの設定と投入
トレーニングスクリプトのハイパーパラメータチューニングを行うために、スイープ機能を使用します。スイープジョブを設定するには、以下のように設定する必要があります：

- compute`： compute`: ジョブを実行する計算対象の名前。
- sampling_algorithm`： 探索空間で使用するハイパーパラメータのサンプリングアルゴリズム。許可される値は `random`、`grid` および `bayesian` である。
- primary_metric`： 各トライアルジョブが報告するプライマリメトリクスの名前。このメトリックは `mlflow.log_metric()` を使用して、ユーザーのトレーニングスクリプトに同じメトリック名で記録されている必要がある。
- ゴール`： primary_metric` の最適化目標。指定できる値は `maximize` と `minimize` である。
- limit`: 掃引ジョブの制限値： スイープジョブの制限値。例えば、トレーニングする試行回数やモデルの最大数など。

コマンドジョブはスイープジョブのベースとして使用されることに注意してください。コマンドジョブの設定はスイープジョブで再利用されます。



In [10]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [11]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/quiet_moon_6qzx0m40gb?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.